In [116]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [143]:
data = pd.read_csv(r'Main_data.csv')
data.head()

,Unnamed: 0,Age,Sex,Resting_BP,Cholestrol_mg/dl,Fasting_sugar,Resting_ECG,Max_HR_achieved,Exercise_induced_angina,ST_depression,Chest_pain_type_1,Chest_pain_type_2,Chest_pain_type_3,Chest_pain_type_4,Diagnosis
0,0,63,1,145,233,1,2,150,0,2.3,1,0,0,0,0
1,1,67,1,160,286,0,2,108,1,1.5,0,0,0,1,1
2,2,67,1,120,229,0,2,129,1,2.6,0,0,0,1,1
3,3,37,1,130,250,0,0,187,0,3.5,0,0,1,0,0
4,4,41,0,130,204,0,2,172,0,1.4,0,1,0,0,0


In [144]:
data.shape

(564, 15)

In [145]:
data.isnull().sum()

Unnamed: 0                 0
Age                        0
Sex                        0
Resting_BP                 0
Cholestrol_mg/dl           0
Fasting_sugar              0
Resting_ECG                0
Max_HR_achieved            0
Exercise_induced_angina    0
ST_depression              0
Chest_pain_type_1          0
Chest_pain_type_2          0
Chest_pain_type_3          0
Chest_pain_type_4          0
Diagnosis                  0
dtype: int64

In [119]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X = data.iloc[:, 0:10].values
y = data.iloc[:,10].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train_t, X_val, y_train_t, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state=0)
sc = StandardScaler()
sc.fit(X_train_t)
X_train_t_std = sc.transform(X_train_t)
X_val_std = sc.transform(X_val)
X_test_std = sc.transform(X_test)

In [120]:
param_grid = [
  {'C': np.array([0.001,0.003,0.01,0.03,0.1,0.3,1,3]), 'kernel': ['linear','rbf','sigmoid'],
  'gamma': np.array([0.001,0.003,0.01,0.03,0.1,0.3,1,3])}
 ]
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
svm_clf = SVC()
grid = GridSearchCV(svm_clf, param_grid, cv=10, scoring='accuracy')

In [121]:
grid.fit(X_train_t_std, y_train_t)

GridSearchCV(cv=10, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'C': array([  1.00000e-03,   3.00000e-03,   1.00000e-02,   3.00000e-02,
         1.00000e-01,   3.00000e-01,   1.00000e+00,   3.00000e+00]), 'kernel': ['linear', 'rbf', 'sigmoid'], 'gamma': array([  1.00000e-03,   3.00000e-03,   1.00000e-02,   3.00000e-02,
         1.00000e-01,   3.00000e-01,   1.00000e+00,   3.00000e+00])}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [122]:
grid.grid_scores_

[mean: 0.59172, std: 0.00713, params: {'C': 0.001, 'gamma': 0.001, 'kernel': 'linear'},
 mean: 0.59172, std: 0.00713, params: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 0.59172, std: 0.00713, params: {'C': 0.001, 'gamma': 0.001, 'kernel': 'sigmoid'},
 mean: 0.59172, std: 0.00713, params: {'C': 0.001, 'gamma': 0.0030000000000000001, 'kernel': 'linear'},
 mean: 0.59172, std: 0.00713, params: {'C': 0.001, 'gamma': 0.0030000000000000001, 'kernel': 'rbf'},
 mean: 0.59172, std: 0.00713, params: {'C': 0.001, 'gamma': 0.0030000000000000001, 'kernel': 'sigmoid'},
 mean: 0.59172, std: 0.00713, params: {'C': 0.001, 'gamma': 0.01, 'kernel': 'linear'},
 mean: 0.59172, std: 0.00713, params: {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'},
 mean: 0.59172, std: 0.00713, params: {'C': 0.001, 'gamma': 0.01, 'kernel': 'sigmoid'},
 mean: 0.59172, std: 0.00713, params: {'C': 0.001, 'gamma': 0.029999999999999999, 'kernel': 'linear'},
 mean: 0.59172, std: 0.00713, params: {'C': 0.001, 'gamma': 0.0299

In [123]:
grid.best_params_

{'C': 3.0, 'gamma': 0.0030000000000000001, 'kernel': 'rbf'}

In [124]:
svm_clf = SVC(C=3, kernel ='rbf', gamma = 0.003)
svm_clf.fit(X_train_t_std, y_train_t)
y_pred_val = svm_clf.predict(X_val_std)
accuracy_score(y_pred_val, y_val)

0.80530973451327437

In [125]:
y_pred_test = svm_clf.predict(X_test_std)
accuracy_score(y_pred_test, y_test)

0.86725663716814161

In [126]:
X_test_std[0,:]

array([-0.98266923,  0.67130006, -1.08562805, -0.67456947,  0.86184676,
       -0.3398699 , -0.74312487,  0.70679665, -0.66666667, -0.76089192])

In [127]:
X_test[0,:]

array([  42.,    1.,    2.,  120.,  295.,    0.,    0.,  162.,    0.,    0.])

In [128]:
y_pred_test[0]

0

In [129]:
y_test[0]

0

array([[-0.8762816 ,  0.67130006,  0.97007101, ..., -1.13864849,
         1.5       ,  1.51102919],
       [-0.76989396,  0.67130006,  0.97007101, ...,  1.05831001,
        -0.66666667, -0.76089192],
       [-1.62099505,  0.67130006, -0.05777852, ...,  1.14618835,
        -0.66666667, -0.76089192],
       ..., 
       [ 1.4642464 ,  0.67130006,  0.97007101, ..., -1.35834434,
         1.5       ,  0.14787652],
       [ 1.88979695, -1.48964681, -2.11347758, ...,  0.22346578,
        -0.66666667,  0.87489128],
       [-1.08905687, -1.48964681, -1.08562805, ..., -0.17198675,
        -0.66666667, -0.76089192]])

In [132]:
nominal_feature = pd.get_dummies(data['Chest_pain_type'], prefix='Chest_pain_type')
data[nominal_feature.columns] = nominal_feature

In [134]:
data.__delitem__('Chest_pain_type')

In [135]:
output_df = data["Diagnosis"]

In [136]:
data.__delitem__("Diagnosis")

In [140]:
data["Diagnosis"] = output_df

In [151]:
data.to_csv("Main_Data.csv")

In [150]:
data.__delitem__("Unnamed: 0")